# User Labeling

## Présentation
Le script en question a pour objectif de labéliser les utilisateurs en deux catégories distinctes : ceux qui sont considérés comme atypiques (avec un label à 1) et ceux qui ne le sont pas (avec un label à 0). 

Les utilisateurs jugés atypiques pourront être identifiés grâce à des comportements ou des actions qui sortent de l'ordinaire

En labélisant les utilisateurs de cette manière, le script peut permettre de mieux comprendre les comportements des utilisateurs.

Comment fonctionne-t-il ?
- Dès qu'un des attributs de l'utilisateur dépasse la valeur minimale qui lui est associée, celui-ci est alors considéré comme atypique.

## Le code
On importe les différentes bibliothèques nécessaires.

In [ ]:
import pymongo
import math

Connexion à la base de données MongoDB

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
user_collection = db["users"]
user_labeled_collection = db["users_labeled"]

On supprime toute la collection des utilisateurs labellisés pour supprimer par la même occasion les données qu'elle contient.

In [ ]:
user_labeled_collection.drop()

On définit les valeurs minimales des champs sur lesquels on va déterminer si un utilisateur est atypique ou non

In [ ]:
min_values_suspicious = {
    'hashtag_frequency': 5,
    'favourites_count': 60_000,
    'friends_count': 6_000,
    'followers_count': 9_000,
    'ratio_friends_followers': 10,
    'tweet_frequency': 70,
    'Ap': 0.02,
    'ratio_sensitive_tweets': 0,
    'ratio_punctuation_tweets': 0.5,
    'visibility': 0.5
}

Définition de la fonction qui détermine le score de suspicion d'un utilisateur

In [ ]:
def get_suspicious_score(user):
    suspicious_fields = []
    for key, value in min_values_suspicious.items():
        if user[key] > value:
            suspicious_fields.append(key)
    return (len(suspicious_fields), suspicious_fields)

Définition de la fonction qui détermine le label d'un utilisateur enfocntion de on score de suspicion

In [ ]:
def get_label(user):
    suspicious_score, suspicious_fields = get_suspicious_score(user)
    if suspicious_score >= 1:
        return (suspicious_score, suspicious_fields, 1)
    else:
        return (suspicious_score, suspicious_fields, 0)

Définition du nombre d'utilisateur à labéliser

In [ ]:
nb_users_labeled = 40_000

On insérer dans mongodb les utilisateurs avec leur label calculé

In [ ]:
users = user_collection.find({}).limit(nb_users_labeled)

users_with_label = []
for user in users:
    suspicious_score, suspicious_fields, label = get_label(user)
    user['label'] = label
    user['suspicious_score'] = suspicious_score
    user['suspicious_fields'] = suspicious_fields
    users_with_label.append(user)

user_labeled_collection.insert_many(users_with_label)

## Quelques statistiques

Affichage de l'occurence de chaque attribut atypique

In [ ]:
pipeline = [
    {
        '$unwind': '$suspicious_fields'
    },
    {
        '$group': {
            '_id': '$suspicious_fields',
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'count': -1}
    }
]

In [ ]:
results = list(user_labeled_collection.aggregate(pipeline))
for result in results:
    print(result['_id'], ':', result['count'])

Regarder la répartition des points de suspicion

In [ ]:
res = user_labeled_collection.aggregate([
    {
        "$group": {
            "_id": "$suspicious_score",
            "count": { "$sum": 1 }
        }
    },
    { 
        "$sort" : { 
            "count" : -1 
        } 
    }
])

for doc in res:
    print(doc['_id'], ':', doc['count'])

Regarder la répartition des labels atypique ou non

In [ ]:
res = user_labeled_collection.aggregate([
    {
        "$group": {
            "_id": "$label",
            "count": { "$sum": 1 }
        }
    },
    { 
        "$sort" : { 
            "count" : -1 
        } 
    }
])

for doc in res:
    print('classe', doc['_id'], ':', doc['count'])